# HuggingFace 커스텀 프로젝트
- model(klue/ber-base)를 활용하여 NSMC(Naver Sentiment Movie Corpus) task 수행
- 데이터: https://github.com/e9t/nsmc
- model: https://huggingface.co/klue/bert-base

In [1]:
import tensorflow
import numpy as np
import transformers
import datasets

print(tensorflow.__version__)
print(np.__version__)
print(transformers.__version__)
print(datasets.__version__)

2.6.0
1.21.4
4.11.3
1.14.0


## Dataset

### Huggingface dataset에서 불러오기
NSMC 데이터셋 https://huggingface.co/datasets/Blpeng/nsmc

In [2]:
from datasets import load_dataset

ds = load_dataset("Blpeng/nsmc")

Using custom data configuration Blpeng___nsmc-55757a98c8abea78
Reusing dataset csv (/aiffel/.cache/huggingface/datasets/csv/Blpeng___nsmc-55757a98c8abea78/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
ds

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'id', 'document', 'label'],
        num_rows: 400000
    })
})

train 데이터만 존재 -> 데이터 분할 필요

In [4]:
train = ds['train']
cols = train.column_names
for i in range(5):
    for col in cols:
        print(col, ":", train[col][i])
    print('\n')

Unnamed: 0 : 0
id : 8112052
document : 어릴때보고 지금다시봐도 재밌어요ㅋㅋ
label : 1


Unnamed: 0 : 1
id : 8132799
document : 디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업이 부러웠는데. 사실 우리나라에서도 그 어려운시절에 끝까지 열정을 지킨 노라노 같은 전통이있어 저와 같은 사람들이 꿈을 꾸고 이뤄나갈 수 있다는 것에 감사합니다.
label : 1


Unnamed: 0 : 2
id : 4655635
document : 폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.
label : 1


Unnamed: 0 : 3
id : 9251303
document : 와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런게 진짜 영화지
label : 1


Unnamed: 0 : 4
id : 10067386
document : 안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.
label : 1




### 결측치 제거

In [5]:
# 결측치를 제거하는 함수 정의
def remove_missing_values(data):
    # 모든 값이 존재하는지 확인
    return all(value is not None for value in data.values())

# train, test 데이터셋에서 결측치 제거
dataset = train.filter(remove_missing_values)

Loading cached processed dataset at /aiffel/.cache/huggingface/datasets/csv/Blpeng___nsmc-55757a98c8abea78/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a/cache-7ae5fa8d6b787a51.arrow


In [6]:
dataset

Dataset({
    features: ['Unnamed: 0', 'id', 'document', 'label'],
    num_rows: 399984
})

약 16개의 데이터 제거됨

> Q. tokeinzer에 전처리 과정도 포함되어 있는 거 아닌가? 왜 결측치가 있는 데이터는 못 받지?  
A. 텍스트 데이터를 모델이 이해할 수 있는 형식으로 변환하는 작업은 가능하지만, 결측치 제거와 같은 데이터 전처리 과정은 포함되어 있지 않음

## Tokenizer & Model
klue/ber-base 모델 https://huggingface.co/klue/bert-base  
Auto Classes 활용

In [7]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
model = AutoModelForSequenceClassification.from_pretrained("klue/bert-base")

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

### 토큰화

In [8]:
def transform(data):
    return tokenizer(
        data['document'],
        truncation = True,
        padding = 'max_length',
        return_token_type_ids = False,    # binary classificaiton task에는 필요 없음
        )

tokenized_dataset = dataset.map(transform)

  0%|          | 0/399984 [00:00<?, ?ex/s]

In [9]:
tokenized_dataset

Dataset({
    features: ['Unnamed: 0', 'id', 'document', 'label', 'input_ids', 'attention_mask'],
    num_rows: 399984
})

### 데이터 분할

In [10]:
train_validtest = tokenized_dataset.train_test_split(test_size=0.2)
valid_test = train_validtest['test'].train_test_split(test_size=0.5)

train_data = train_validtest["train"]
valid_data = valid_test["train"]
test_data = valid_test["test"]

In [11]:
train_data

Dataset({
    features: ['Unnamed: 0', 'id', 'document', 'label', 'input_ids', 'attention_mask'],
    num_rows: 319987
})

## Trainer

### 학습 준비

In [12]:
# 학습 관련 설정을 미리 지정
from transformers import Trainer, TrainingArguments

output_dir = './outputs'

training_arguments = TrainingArguments(
    output_dir,                             # output이 저장될 경로
    evaluation_strategy="epoch",           #evaluation하는 빈도
    learning_rate = 2e-5,                         #learning_rate
    per_device_train_batch_size = 16,   # 각 device 당 batch size
    per_device_eval_batch_size = 8,    # evaluation 시에 batch size
    num_train_epochs = 1,                     # train 시킬 총 epochs
    weight_decay = 0.01,                        # weight decay
)

In [13]:
from datasets import load_metric
metric = load_metric("accuracy")

# binary classification
def compute_metrics(eval_pred):    
    predictions,labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references = labels)

### 학습

In [27]:
import os
import wandb

os.environ["WANDB_DISABLED"] = "true"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [28]:
trainer = Trainer(
    model=model,           # 학습시킬 model
    args=training_arguments,           # TrainingArguments을 통해 설정한 arguments
    train_dataset=train_data,    # training dataset
    eval_dataset=valid_data,       # evaluation dataset
    compute_metrics=compute_metrics,
)
trainer.train()

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


AssertionError: WandbCallback requires wandb to be installed. Run `pip install wandb`.

In [ ]:
# 평가
trainer.evaluate(test_data)

## wandb와 함께 사용
- https://docs.wandb.ai/guides/integrations/huggingface
- https://wandb.ai/matt24/vit-snacks-sweeps/reports/Hyperparameter-Search-for-HuggingFace-Transformer-Models--VmlldzoyMTUxNTg0

In [22]:
!pip install wandb

In [ ]:
import wandb

wandb.login()

In [ ]:
# method
sweep_config = {
    'method': 'random'
}

# hyperparameters
parameters_dict = {
    'epochs': {
        'values': [1, 2]
        },
    'batch_size': {
        'values': [8, 16, 32]
        },
    'learning_rate': {
        'distribution': 'log_uniform_values',
        'min': 1e-5,
        'max': 1e-3
    },
    'weight_decay': {
        'values': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]
    },
}

sweep_config['parameters'] = parameters_dict


In [ ]:
def train(config=None):
    with wandb.init(config=config):
        config = wandb.config
        
        wandb_training_arguments = TrainingArguments(
            output_dir='./sweeps',                             # output이 저장될 경로
            evaluation_strategy="epoch",           #evaluation하는 빈도
            learning_rate = config.learning_rate,                         #learning_rate
            per_device_train_batch_size = config.batch_size,   # 각 device 당 batch size
            per_device_eval_batch_size = 8,    # evaluation 시에 batch size
            num_train_epochs = config.epochs,                     # train 시킬 총 epochs
            weight_decay = config.weight_decay,                        # weight decay
            report_to="wandb",  # enable logging to W&B
            logging_steps=1,  # how often to log to W&B
        )
        
        model = AutoModelForSequenceClassification.from_pretrained("klue/bert-base")

        trainer = Trainer(
            model=model,           # 학습시킬 model
            args=wandb_training_arguments,           # TrainingArguments을 통해 설정한 arguments
            train_dataset=train_data,    # training dataset
            eval_dataset=valid_data,       # evaluation dataset
            compute_metrics=compute_metrics,
        )
        
        trainer.train()

In [ ]:
sweep_id = wandb.sweep(sweep_config, project='nsmc_klue')

wandb.agent(sweep_id, train, count=1)

In [ ]:
with wandb.init(project="nsmc_klue") as run:
    # Pass the name and version of Artifact
    my_model_name = "<run_name>:latest"
    my_model_artifact = run.use_artifact(my_model_name)

    # Download model weights to a folder and return the path
    model_dir = my_model_artifact.download()

    # Load your Hugging Face model from that folder
    #  using the same model class
    model = AutoModelForSequenceClassification.from_pretrained(
        model_dir, num_labels=num_labels
    )